# Histopathology Pipeline Prototype (2024년 Biopsy Matches)

이 노트북은 2024년 biopsy matching 데이터셋을 탐색하고 histopathology classifier 학습을 준비하는 전체 흐름을 정리한 문서입니다. raw diagnostic report를 machine-readable label로 정제하고, `.svs` whole-slide image(WSI)를 기반으로 한 patch extraction 절차를 개괄하는 것이 핵심입니다.

## 목표

1. Excel/CSV에서 export된 biopsy metadata를 점검하고 정제합니다.
2. 이질적인 diagnostic 텍스트를 일관된 disease label로 통합합니다.
3. 이용 가능한 annotation을 바탕으로 실용적인 modelling target을 정의합니다.
4. downstream modelling에 사용할 stratified train/validation/test split을 생성합니다.
5. `.svs` slide에 대한 patch extraction workflow를 개괄하고, 누락된 파일 여부와 tissue filtering heuristic을 설명합니다.

> **참고:** 현재 레포지토리에는 metadata CSV만 포함되어 있습니다. 실제 WSI `.svs` 파일은 patch extractor 셀을 실행하기 전에 별도로 마운트해야 합니다.

In [ ]:

from __future__ import annotations

# --- 데이터 적재 및 기본 환경 설정 ---
# 공통 모듈을 한 번에 import하고, 이후 단계에서 반복 사용되는 기본 DataFrame을 메모리에 적재합니다.
# 이렇게 구성하면 노트북 전체에서 동일한 상태를 공유하며 중복 I/O를 줄일 수 있습니다.

import json
import re
from collections import Counter
from dataclasses import dataclass
from pathlib import Path
from typing import Iterable, List, Optional

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

DATA_PATH = Path('Data') / '조직검사 결과 매칭(2024)_utf8_pruned.csv'
# CSV 경로를 상수로 두어 실험 환경에 따라 경로만 교체하면 되도록 했습니다.
assert DATA_PATH.exists(), f"Missing dataset: {DATA_PATH}"
# 조기 실패를 통해 경로 설정 오류를 빠르게 발견하고 후속 셀의 예외 연쇄를 막습니다.

raw_df = pd.read_csv(DATA_PATH)
# 모든 후속 통계와 split 단계가 동일한 원본 뷰에서 출발하도록 raw_df를 단일 소스로 유지합니다.
print(f"Loaded {len(raw_df):,} biopsy records with {raw_df.shape[1]} columns")
raw_df.head()


### 결측값 개요

이 단계에서는 `raw_df`의 모든 컬럼에 대해 결측값 비율을 계산하고 내림차순으로 정렬합니다. 데이터 품질을 빠르게 점검하고, 추가 정제가 필요한 필드를 선별하기 위한 사전 작업입니다.

셀을 실행하면 결측 비율이 가장 높은 컬럼부터 나열된 `Series`가 출력됩니다. 상위 항목을 확인해 결측이 심한 열을 별도로 보강할지, 분석에서 제외할지 판단하세요.

In [ ]:

# --- 결측값 진단으로 전처리 우선순위 설정 ---
# 각 컬럼의 결측 비율을 계산해 downstream 단계에서 먼저 다듬어야 할 필드를 빠르게 파악합니다.
# 비율 기반 정렬을 사용하면 전체 데이터 크기에 상관없이 상대적 우선순위를 즉시 확인할 수 있습니다.

missing_summary = (
    raw_df.isna()
    .mean()
    .sort_values(ascending=False)
    .to_frame(name='missing_ratio')
)
# 상위 열만 확인해도 주요 결측 패턴을 파악할 수 있어 탐색 시간을 단축합니다.
missing_summary.head(10)


## Diagnosis 정규화

`DIAGNOSIS` 컬럼에는 장문의 free-text 문자열이 포함되어 있어 모델 학습에 바로 활용하기 어렵습니다. 아래 단계는 텍스트를 간결한 label로 변환하여 클래스 수를 제어하고, downstream task에서 일관된 정답을 제공하기 위한 전처리입니다.

1. 첫 번째 쉼표 이전 부분만 유지하여 staging·margin과 같은 부가 설명을 제거합니다.
2. 괄호나 중복 공백을 제거해 문자열 패턴을 단순화합니다.
3. 소문자로 변환한 뒤 자주 등장하는 동의어를 canonical form으로 매핑합니다.

코드를 실행하면 규칙 기반 매핑이 적용된 `raw_df['target_label']`이 생성됩니다. 각 규칙의 적용 결과를 검토해 잘못된 매핑이 없는지 확인하고, 필요 시 목록을 보완하세요.

In [ ]:

# --- 규칙 기반 진단명 정규화 ---
# 질병명을 사전 정의된 canonical label로 매핑해 희귀 변형을 통합하고 통계적 안정성을 확보합니다.
# 정규표현식을 사용하면 접두어나 수식어가 추가된 케이스도 동일 로직으로 관리할 수 있습니다.

NORMALISATION_RULES = [
    (r'^mast cell tumor', 'mast cell tumor'),
    (r'^cutaneous mast cell tumor', 'mast cell tumor'),
    (r'^subcutaneous mast cell tumor', 'mast cell tumor'),
    (r'^mammary gland adenoma', 'mammary adenoma'),
    (r'^mammary complex adenoma', 'mammary complex adenoma'),
    (r'^mammary benign mixed tumor', 'mammary benign mixed tumor'),
    (r'^mammary carcinoma', 'mammary carcinoma'),
    (r'^mammary duct carcinoma', 'mammary carcinoma'),
    (r'^mammary adenoma', 'mammary adenoma'),
    (r'^lipoma', 'lipoma'),
    (r'^subcutaneous lipoma', 'lipoma'),
    (r'^hepatic lipoma', 'lipoma'),
    (r'^sebaceous adenoma', 'sebaceous adenoma'),
    (r'^sebaceous epithelioma', 'sebaceous epithelioma'),
    (r'^soft tissue sarcoma', 'soft tissue sarcoma'),
    (r'^trichoblastoma', 'trichoblastoma'),
]


def normalise_diagnosis(value: str) -> str:
    if not isinstance(value, str) or not value.strip():
        return 'unknown'

    # 쉼표 앞부분만 남겨 병기/절제여부 등 보조 정보를 제거해 핵심 진단명에 집중합니다.
    base = value.split(',')[0]
    # 괄호와 특수문자를 걷어내 텍스트 패턴을 단순화하고 규칙 재사용성을 높입니다.
    base = re.sub(r'\([^)]*\)', '', base)
    base = re.sub(r'[^a-zA-Z0-9\s]', ' ', base)
    base = re.sub(r'\s+', ' ', base).strip().lower()

    for pattern, canonical in NORMALISATION_RULES:
        if re.match(pattern, base):
            return canonical

    # 사전에 없는 경우에는 정제된 문자열을 그대로 반환해 신규 케이스를 추적할 수 있도록 합니다.
    return base


raw_df['disease_family'] = raw_df['DIAGNOSIS'].map(normalise_diagnosis)
# 정규화된 family 레벨을 별도 컬럼으로 유지해 threshold 조정이나 규칙 보강이 용이하도록 했습니다.
print('Unique disease families:', raw_df['disease_family'].nunique())
raw_df['disease_family'].value_counts().head(20)


### 모델링 target 선택

데이터셋에는 수천 개의 diagnostic 문구가 존재하며, 모든 label을 개별 클래스로 다루면 극도로 희소해집니다. 따라서 빈도가 낮은 클래스는 `other` 버킷으로 묶고, 가장 흔한 disease family에 집중해 안정적인 분포를 확보합니다.

셀 실행 시 `MIN_CASES` 이상 등장하는 label 개수와 목록이 출력됩니다. 결과를 통해 클래스 불균형 정도를 파악하고, threshold를 조정해 원하는 granularity를 찾을 수 있습니다.

In [ ]:

# --- 희소 클래스 통합 전략 ---
# 클래스 빈도가 극단적으로 낮으면 모델 학습이 불안정해지므로, 최소 증례 수 기준을 둬 주요 질환군에 집중합니다.
# 한 번 계산한 value_counts를 재사용해 threshold를 조정할 때도 비용이 들지 않도록 했습니다.

MIN_CASES = 250
value_counts = raw_df['disease_family'].value_counts()
major_labels = value_counts[value_counts >= MIN_CASES].index.tolist()
print(f"Keeping {len(major_labels)} frequent labels (≥{MIN_CASES} cases)")

raw_df['target_label'] = np.where(
    raw_df['disease_family'].isin(major_labels),
    raw_df['disease_family'],
    'other'
)
# 희귀 클래스는 'other'로 묶어 추후 분석 시 다시 drill-down 할 수 있는 최소 정보를 남겨둡니다.

raw_df['target_label'].value_counts()


### Train/validation/test split 메타데이터

정규화된 target label을 기준으로 stratified split을 생성하여 추후 patch-level 데이터와 결합합니다. 동일 환자·슬라이드가 여러 split에 중복되지 않도록 메타데이터 키(`INSP_RQST_NO`, `FOLDER`, `FILE_NAME`)를 유지합니다.

코드를 실행하면 `train_df`, `valid_df`, `test_df`가 만들어지고, 각 데이터프레임의 크기가 로그로 표시됩니다. 분할된 데이터는 모델 학습에서 재현성을 보장하고, validation/test 평가를 공정하게 유지하는 데 사용됩니다.

In [ ]:

# --- Stratified 데이터 분할 ---
# 동일한 환자/슬라이드가 여러 split에 중복되지 않도록 키 컬럼만 남겨 안정적인 평가 집합을 만듭니다.
# Stratified split을 통해 label 분포를 최대한 유지해 모델 튜닝 시 분산을 줄입니다.

META_COLUMNS = ['INSP_RQST_NO', 'FOLDER', 'FILE_NAME', 'target_label']
meta_df = raw_df[META_COLUMNS].drop_duplicates().reset_index(drop=True)

train_df, temp_df = train_test_split(
    meta_df,
    test_size=0.3,
    random_state=42,
    stratify=meta_df['target_label']
)
valid_df, test_df = train_test_split(
    temp_df,
    test_size=0.5,
    random_state=42,
    stratify=temp_df['target_label']
)

print('Train size:', len(train_df))
print('Valid size:', len(valid_df))
print('Test size :', len(test_df))

split_summary = {
    'train': train_df['target_label'].value_counts(normalize=True).to_dict(),
    'valid': valid_df['target_label'].value_counts(normalize=True).to_dict(),
    'test': test_df['target_label'].value_counts(normalize=True).to_dict(),
}
# JSON 구조로 직렬화하면 버전 관리나 외부 파이프라인 연동 시 재사용이 수월합니다.
json.dumps(split_summary, indent=2)


## WSI 가용성 확인

metadata는 요청별 `FOLDER`에 저장된 `.svs` 파일을 참조합니다. 실제 슬라이드 파일이 마운트된 디렉터리 경로를 `WSI_ROOT`에 지정해 존재 여부를 점검합니다.

경로가 존재하지 않으면 경고 메시지가 출력되며, patch extraction 단계가 자동으로 건너뛰어집니다. 반대로 슬라이드가 감지되면 이후 셀에서 곧바로 patch를 추출할 수 있습니다.

In [ ]:

# --- 슬라이드 파일 가용성 점검 ---
# metadata에서 참조하는 경로 구조를 실제 파일 시스템과 비교해, 추후 patch 추출 단계에서 발생할 실패를 미리 차단합니다.

WSI_ROOT = Path('Data/WSI')  # TODO: update to actual location

if not WSI_ROOT.exists():
    print(f"WSI root missing at {WSI_ROOT.resolve()}. Patch extraction will be skipped.")
else:
    sample_rows = raw_df.head(3)
    for _, row in sample_rows.iterrows():
        candidate = WSI_ROOT / row['FOLDER'] / row['FILE_NAME']
        # 샘플 경로를 출력해 슬라이드 구조가 예상과 일치하는지 수동으로 확인할 수 있게 합니다.
        print(candidate, 'exists' if candidate.exists() else 'missing')


## Patch extraction 유틸리티

고해상도 패치를 스트리밍하기 위해 [OpenSlide](https://openslide.org/)를 활용합니다. prerequisites는 다음과 같습니다.

```bash
sudo apt-get install openslide-tools
pip install openslide-python
```

라이브러리가 정상적으로 import되면 `HAS_OPENSLIDE` 플래그가 `True`로 설정되고, 이후 셀에서 실제 patch extraction 함수가 작동합니다. 설치가 되지 않았을 경우 우회 메시지를 출력해 추후 조치를 안내합니다.

In [ ]:

# --- OpenSlide 의존성 로딩 ---
# WSI 스트리밍은 외부 C 라이브러리에 의존하므로, import 단계에서 명시적으로 가용성을 점검해 이후 셀의 예외를 최소화합니다.

try:
    import openslide
    from PIL import Image
    HAS_OPENSLIDE = True
    print('OpenSlide version:', openslide.__library_version__)
except Exception as exc:  # noqa: BLE001
    HAS_OPENSLIDE = False
    print('OpenSlide not available:', exc)


In [ ]:

# --- Patch 추출 유틸리티 정의 ---
# 추후 재현 가능한 patch index를 만들기 위해 좌표, 레벨, 레이블 정보를 하나의 dataclass로 묶어 구조화합니다.
# 각 함수에는 실패 지점을 조기에 드러내기 위한 방어 로직을 추가해 대규모 배치 실행 시 디버깅 비용을 줄입니다.

@dataclass
class PatchSpec:
    slide_id: str
    x: int
    y: int
    level: int
    size: int
    label: str
    source_path: Path


def resolve_slide_path(row: pd.Series, root: Path) -> Optional[Path]:
    """Resolve the absolute slide path using folder/name hints."""
    # 일부 항목은 폴더 구조가 다를 수 있으므로, 여러 후보 경로를 순차적으로 검사해 누락을 최소화합니다.
    candidates = [root / row['FOLDER'] / row['FILE_NAME'], root / row['FILE_NAME']]
    for cand in candidates:
        if cand.exists():
            return cand
    return None


def tissue_fraction(tile: Image.Image) -> float:
    # 간단한 intensity 기반 휴리스틱으로 배경을 제거해, 계산 비용이 큰 deep model을 호출하기 전 1차 필터링을 수행합니다.
    arr = np.asarray(tile.convert('L'))
    norm = (arr - arr.min()) / (arr.ptp() + 1e-6)
    return float((norm < 0.85).mean())


def extract_patches_for_slide(
    slide_path: Path,
    label: str,
    level: int = 0,
    patch_size: int = 256,
    stride: Optional[int] = None,
    max_patches: int = 200,
    tissue_threshold: float = 0.2,
) -> List[PatchSpec]:
    if not HAS_OPENSLIDE:
        raise RuntimeError('OpenSlide support is required to extract patches.')

    slide = openslide.OpenSlide(str(slide_path))
    stride = stride or patch_size
    width, height = slide.level_dimensions[level]

    patches: List[PatchSpec] = []
    for y in range(0, height - patch_size + 1, stride):
        for x in range(0, width - patch_size + 1, stride):
            region = slide.read_region((x, y), level, (patch_size, patch_size))
            # tissue_fraction이 낮으면 배경 패치이므로 스토리지와 학습 시간을 아끼기 위해 건너뜁니다.
            if tissue_fraction(region) < tissue_threshold:
                continue
            patches.append(PatchSpec(
                slide_id=slide_path.stem,
                x=x,
                y=y,
                level=level,
                size=patch_size,
                label=label,
                source_path=slide_path,
            ))
            if len(patches) >= max_patches:
                break
        if len(patches) >= max_patches:
            break
    slide.close()
    return patches


def build_patch_index(
    split_df: pd.DataFrame,
    root: Path,
    level: int = 0,
    patch_size: int = 256,
    stride: Optional[int] = None,
    max_patches_per_slide: int = 200,
    tissue_threshold: float = 0.2,
) -> pd.DataFrame:
    records = []
    for _, row in split_df.iterrows():
        slide_path = resolve_slide_path(row, root)
        if slide_path is None:
            continue
        try:
            patches = extract_patches_for_slide(
                slide_path=slide_path,
                label=row['target_label'],
                level=level,
                patch_size=patch_size,
                stride=stride,
                max_patches=max_patches_per_slide,
                tissue_threshold=tissue_threshold,
            )
        except Exception as exc:  # noqa: BLE001
            # 실패한 슬라이드는 로깅만 하고 계속 진행해 배치 실행이 중단되지 않도록 합니다.
            print(f"Skipping {row['FILE_NAME']}: {exc}")
            continue

        for patch in patches:
            records.append({
                'slide_id': patch.slide_id,
                'x': patch.x,
                'y': patch.y,
                'level': patch.level,
                'size': patch.size,
                'label': patch.label,
                'source_path': str(patch.source_path),
            })

    return pd.DataFrame.from_records(records)


### Patch extraction 데모

이 셀은 training split에 속한 슬라이드 중 일부를 대상으로 작은 patch index를 구축하는 예시입니다. 실제 파일과 OpenSlide 환경이 갖춰져 있다면 지정된 좌표와 크기에 맞춰 패치를 추출하고, 간단한 품질 필터를 적용해 parquet 파일로 저장하는 흐름을 시연합니다.

실행 결과:
- OpenSlide 또는 슬라이드 디렉터리가 없으면 원인을 설명하는 메시지가 출력됩니다.
- 모든 준비가 되어 있으면 추출된 패치 수와 저장 경로가 로그로 표시됩니다. 이를 통해 pipeline 구동 여부를 즉시 확인할 수 있습니다.

In [ ]:

# --- Patch extraction 데모 실행 제어 ---
# 환경 준비 여부에 따라 실행을 분기해, 의존성이 없을 때도 노트북 전체 흐름을 중단하지 않고 안내 메시지를 제공합니다.

if not HAS_OPENSLIDE:
    print('OpenSlide missing — skipping patch extraction demo.')
elif not WSI_ROOT.exists():
    print('WSI directory not found — mount slides and re-run.')
else:
    demo_df = train_df.head(2)
    patch_index = build_patch_index(
        split_df=demo_df,
        root=WSI_ROOT,
        level=0,
        patch_size=256,
        stride=256,
        max_patches_per_slide=16,
        tissue_threshold=0.25,
    )
    print('Extracted', len(patch_index), 'patches')
    patch_index.head()


## 다음 단계

* 새로운 diagnostic variant가 발견되면 정규화 규칙을 업데이트하여 label 누락을 방지합니다.
* downstream training script에서 사용할 수 있도록 split metadata(`train.csv`, `valid.csv`, `test.csv`)와 생성된 `patch_index.parquet`를 버전 관리합니다.
* 모델 입력 전에 patch-level filtering(예: blur detection, stain normalisation)을 적용해 데이터 품질을 향상시킵니다.
* 반복적인 patch extraction을 방지하도록 slide-level cache 또는 artifact 저장 전략을 마련합니다.